In [3]:
import numpy as np
import pandas as pd
import os
import argparse
import pickle
import torch
import json
import torch.nn as nn
from torch.utils.data import DataLoader
from esim.data import NLIDataset
from esim.model import ESIM
from esim.utils_train import train, validate

In [4]:
from esim.data_reader import Preprocessor
from esim.data_reader import NLIDataset
import pandas as pd

In [5]:
from esim.data_reader import data_process

In [10]:
train_data = pd.read_csv('../data/train_data20.csv')
train_data=train_data.sample(frac=1).reset_index(drop=True)
test_data=pd.read_csv('../data/test_data.csv')
processor=Preprocessor()
train_data=processor.seg_data(train_data)
test_data=processor.seg_data(test_data)
word_set=processor.set_data(train_data)|processor.set_data(test_data)

embeddings_index=processor.read_embedding()
##词向量原始10000条与新数据的并集为新的embedingmatrix的索引
word_set=set([i for i in embeddings_index.keys()][:10000])|word_set

embedding_matrix,word_to_indices=processor.build_embedding_matrix(word_set,embeddings_index)

train_esim=processor.gen_input_data(train_data,word_to_indices)
valid_esim=processor.gen_input_data(test_data,word_to_indices)

Preparing embeddings matrix...
12257
12259
第一届
扎得
前金院
几年
部该
是旦
花衫
取个
多吃生
银保
心智健全
选拔干部
几首
一步
计财部
本特
岗平
存点
说水
机是
付额
爸姓
几楼
第一部
说会
武旦
版先
触电时
慢病
喝咖啡
能买个
四片
几句
有延项
做二便
瓷质
几下
15%
五十多岁
爱是
第一件
总支部
需哟
两定
多血
喝多
几个
具体表
若得
一个月
完步
B超
三方
长是
得直
肖总
多个
几句话
想美白
说点
廖俊波
一部
双录
人有
一次
金要
高菁
双查
二十多
混着
挺大
管瘤
党执政
付线
党风廉政
吃生
3T
报灯
三十多岁
中国新民主主义革命
乐一乐
孔庆伟
没换
一届
一台
X射线
讲个
有家
VISA卡
五条
包不包
贷是
我帅
发量
一棵
起付
要少
弃项
卡制作
前应
太牛
不流
我能
能聊
区指
六有
薪易贷
口病
做直
一点点
上能
我姓
好萌
一有
老掉
一首
合要
这点
太干
易使
陈园
针口
我用
几分钟
每分
三甲
人多
几首歌
一步步
严治
加项
没电是
四条
我办
帮个
好几年
期收
好少
爱眼日
五水共治
卖个
第十
跳个
有何
六十多
笑个
两个
三型
需经
能值
天能
四十多岁
不多
说个
扎过
费缓
总吃
相适
事是
诊有
三位
缺碘有
第二批
抵支
合新
一座
病有
好大
另一只
太瘦会
奶及
一句
延项
第十九届
姚军
老顶
爱德
用网
骤停
笨点
外配
哪一点
五十多
真如铁
党应
能报
我码
过轻会
一种
十杯
没掉
五水共治指
讲人话
取卡
后要
二批
一群
几种
四性
一是
二便
三个
几分
术后
做个
要分
你会
两句话
有弃
X光
六个
多岁
没带
卡到
几本书
毛净
孕前
我要
还会
熟食品
下天
姜坏
哪来
加酶
黄宝
完血
一杯
生旦
阔落
前要
中有
安博士
时恒
生角
默比
大是
钱算
五种
几辆
要放
有车
保险期
哪读
不需
最多人
两种
给爷
费由
生是
缴钱
弃检
缓缴
供血
人上
需提交
取号
一岗双责
哪三大
一段
几月
变胖
一指
大规
人一保
最帅
几杯
划账
曲种
我爱你
归宿点
首个
一天
合有
孙建
好多好多
完酸
劳逸
过生日
三严
只会
五百强
睡够
保护环
总起
聊点
C13
一核
我点
叫单

## model

In [11]:
hidden_size = 300
dropout = 0.3
num_classes = 2
epochs = 4
batch_size =256
lr = 0.0004
patience = 5
max_grad_norm = 10
checkpoint = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
save_flag = False

In [12]:
print("\t* Loading training data...")
max_len=20
Ntrain = NLIDataset(train_esim, padding_idx=1, max_premise_length=max_len, max_hypothesis_length=max_len)
Nvalid = NLIDataset(valid_esim, padding_idx=1, max_premise_length=max_len, max_hypothesis_length=max_len)
train_loader = DataLoader(Ntrain, shuffle=False, batch_size=batch_size)
valid_loader = DataLoader(Nvalid, shuffle=False, batch_size=batch_size)
embeddings = torch.tensor(embedding_matrix, dtype=torch.float).to(device)

	* Loading training data...


In [ ]:
save_path='esim_model.pkl'

In [13]:
model = ESIM(embeddings.shape[0],
             embeddings.shape[1],
             hidden_size,
             embeddings=embeddings,
             padding_idx=1,
             dropout=dropout,
             num_classes=num_classes,
             device=device).to(device)

# -------------------- Preparation for training  ------------------- #
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode="max",
                                                       factor=0.5,
                                                       patience=0)

Best_auc=0
# -------------------- Training epochs ------------------- #
print("\n",
      20 * "=",
      "Training ESIM model on device: {}".format(device),
      20 * "=")

patience_counter = 0
for epoch in range(0, epochs+1):
    print("* Training epoch {}:".format(epoch))
    model,epoch_time, epoch_loss, epoch_accuracy = train(model,
                                                   train_loader,
                                                   optimizer,
                                                   criterion,
                                                   epoch,
                                                   max_grad_norm)
    print("-> Training time: {:.4f}s, loss = {:.4f}, accuracy: {:.4f}%"
          .format(epoch_time, epoch_loss, (epoch_accuracy*100)))

    print("* Validation for epoch {}:".format(epoch))   
    _, auc,acc = validate(model, valid_loader,criterion)
    print("\t* Validation auc before training: {:.4f}%, accuracy (0.85): {:.4f}%"
          .format(auc*100, acc*100))
    if auc>Best_auc:
        Best_auc=auc
        torch.save(model,'model.pkl')
    # Update the optimizer's learning rate with the scheduler.
    scheduler.step(epoch_accuracy)

    if patience_counter >= patience:
        print("-> Early stopping: patience limit reached, stopping...")
        break

  0%|          | 0/483 [00:00<?, ?it/s]/home/su/HL/FQA/ESIM/esim/utils.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  sequences_lengths.new_tensor(torch.arange(0, len(sequences_lengths)))
Avg. batch proc. time: 0.1468s, loss: 0.6911:   0%|          | 1/483 [00:00<01:27,  5.50it/s]


 ==================== Training ESIM model on device: cuda:0 ====================
* Training epoch 0:


Avg. batch proc. time: 0.0661s, loss: 0.1996: 100%|██████████| 483/483 [00:34<00:00, 14.12it/s]


-> Training time: 34.1987s, loss = 0.1996, accuracy: 91.5262%
* Validation for epoch 0:


Avg. batch proc. time: 0.0648s, loss: 0.1331:   0%|          | 2/483 [00:00<00:33, 14.39it/s]

...

-> Training time: 34.9508s, loss = 0.0564, accuracy: 98.0383%
* Validation for epoch 3:


Avg. batch proc. time: 0.0676s, loss: 0.0566:   0%|          | 2/483 [00:00<00:34, 13.90it/s]

	* Validation auc before training: 94.9594%, accuracy (0.85): 88.9187%
* Training epoch 4:


Avg. batch proc. time: 0.0688s, loss: 0.0455: 100%|██████████| 483/483 [00:35<00:00, 13.61it/s]


-> Training time: 35.4795s, loss = 0.0455, accuracy: 98.4169%
* Validation for epoch 4:
	* Validation auc before training: 94.0788%, accuracy (0.85): 87.7757%


## 模型存储

In [14]:
#torch.save(model,'model.pkl')
import json 
with open('word_to_indices.json','w') as f:
    json.dump(word_to_indices,f)